# Retrieval augmented generation

If not already done run this in the top level folder:
```
pip install -r requirements.txt
```




In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")

Found Azure OpenAI API Base Endpoint: https://cog-pge4idpaukhle.openai.azure.com/


# Create vector search index

Create your search index schema and vector search configuration

In [2]:
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential

from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndex

)

credential = AzureKeyCredential(os.environ["AZURE_AI_SEARCH_KEY"]) if len(os.environ["AZURE_AI_SEARCH_KEY"]) > 0 else DefaultAzureCredential()

index_name = "movies-semantic-index"

index_client = SearchIndexClient(
    endpoint=os.environ["AZURE_AI_SEARCH_ENDPOINT"], 
    credential=credential
)

# Create a search index with the fields and a vector field which we will fill with a vector based on the overview field
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="genre", type=SearchFieldDataType.String),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="year", type=SearchFieldDataType.String),
    SearchableField(name="rating", type=SearchFieldDataType.String),
    SearchableField(name="plot", type=SearchFieldDataType.String),
    SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile"),
]

# Configure the vector search configuration  
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw"
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        )
    ]
)

# Configure the semantic search configuration to prefer title and tagline fields over overview
semantic_config = SemanticConfiguration(
    name="movies-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        keywords_fields=[SemanticField(field_name="genre")],
        content_fields=[SemanticField(field_name="plot")]
    )
)

# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_search=semantic_search)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 movies-semantic-index created


## Load custom data
Here's how you load custom data into an embedding model and how you use langchain to query it.


In [3]:
import os
import json
from openai import AzureOpenAI
from azure.search.documents import SearchClient

client = AzureOpenAI(
        api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
        api_version = os.getenv("AZURE_OPENAI_VERSION"),
        azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

# use an embeddingsmodel to create embeddings
def get_embedding(text, model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# 1. define function to parse csv row and create embedding for overview text
def parseMovie(movie):
    print(movie)
    return dict([
        ("id", str(movie["movie_id"])),
        ("genre", movie["movie_genre"]),
        ("title", movie["movie_title"]),
        ("year", str(movie["movie_year"])),
        ("rating", str(movie["movie_rating"])),
        ("plot", movie["movie_plot"]),
        ("vector", get_embedding(movie["movie_plot"]))
    ])

# 2. load movies from csv
movies = []
with open('./movies.json') as json_data:
    moviesJson = json.load(json_data)
    line_count = 0
    for movieJson in moviesJson:
        movieEmbedding = parseMovie(movieJson)
        movies.append(movieEmbedding)
        line_count += 1
    print(f'Processed {line_count} lines.')
print('Loaded %s movies.' % len(movies))


# 3. upload documents to vector store
search_client = SearchClient(
    endpoint=os.environ["AZURE_AI_SEARCH_ENDPOINT"], 
    index_name=index_name,
    credential=credential
)

result = search_client.upload_documents(movies)
print(f"Successfully loaded {len(movies)} movies into Azure AI Search index.")



{'movie_id': 1, 'movie_genre': 'Action', 'movie_title': 'The Smonger Games', 'movie_year': 2026, 'movie_rating': 8, 'movie_plot': "In the alien planet Smorgia, a group of Smoorghs are forced to compete in a deadly game called The Smonger Games. The games are organized by the tyrannical ruler of Smorgia, who enjoys watching the competition. Each participant has a unique special ability, and the last survivor will be granted their greatest desires. As the games progress, alliances are formed, betrayals occur, and the true nature of the ruler is revealed. The main character, Smok, discovers that his special ability allows him to control fire. With this power, he becomes a formidable contender and starts a rebellion against the ruler. Smok leads a group of Smoorghs to fight for their freedom and overthrow the oppressive regime. The movie was filmed in the stunning city of Smonopolis and had a budget of 50 million Smorps.The Smonger Games featured 50 actors from different planets, making it

# Query index and create a response

In [4]:
from openai import AzureOpenAI
from azure.search.documents.models import (
    VectorizedQuery
)

client = AzureOpenAI(
        api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
        api_version = os.getenv("AZURE_OPENAI_VERSION"),
        azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
model_name = os.getenv("AZURE_OPENAI_COMPLETION_MODEL")

index_client = SearchClient(
    endpoint=os.environ["AZURE_AI_SEARCH_ENDPOINT"], 
    index_name=index_name,
    credential=credential
)

question = "Tell me about the latest Ant Man movie. When was it released?"

# create a vectorized query based on the question
vector = VectorizedQuery(vector=get_embedding(question), k_nearest_neighbors=5, fields="vector")


# create search client to retrieve movies from the vector store
found_docs = list(search_client.search(
    search_text=None,
    query_type="semantic",
    semantic_configuration_name="movies-semantic-config",
    vector_queries=[vector],
    select=["title", "genre", "plot", "year"],
    top=5
))

found_docs_as_text = " "
# print the found documents and the field that were selected
for doc in found_docs:
    print("Movie: {}".format(doc["title"]))
    print("Genre: {}".format(doc["genre"]))
    print("Year: {}".format(doc["year"]))
    print("----------")
    found_docs_as_text += " "+ "Movie Title: {}".format(doc["title"]) +" "+ "Release Year: {}".format(doc["year"]) + " "+ "Movie Plot: {}".format(doc["plot"])
    
# augment the question with the found documents and ask the LLM to generate a response
system_prompt = "You are an assistant to the user, you are given some context below, please answer the query of the user with as much detail as possible"

parameters = [system_prompt, ' Context:', found_docs_as_text , ' Question:', question]
joined_parameters = ''.join(parameters)

response = client.chat.completions.create(
        model = deployment_name,
        messages = [{"role" : "assistant", "content" : joined_parameters}],
    )

print (response.choices[0].message.content)

Movie: Ant-Man
Genre: Science fiction
Year: 2015
----------
Movie: Ant-Man
Genre: Science Fiction
Year: 2024
----------
Movie: Ant-Man Returns
Genre: Superhero
Year: 2022
----------
Movie: Ant-Man and the Lost City
Genre: Science Fiction
Year: 2023
----------
Movie: Ant-astic
Genre: Action
Year: 2022
----------
The latest Ant-Man movie is titled "Ant-Man and the Lost City." It was released in 2023. In this film, Ant-Man, portrayed by Alex Kendrick, discovers a hidden city inhabited by an ancient civilization of miniature beings called the Microns after emerging from the Quantum Realm. The plot involves Ant-Man's efforts to protect the Microns and their powerful energy source from a ruthless businessman, played by Laurence Fishburne, who wants to exploit it for his own purposes. The movie is filled with mind-bending twists, epic action sequences, and highlights Ant-Man's heroism and abilities. The intricate design and scale of the Micron city were brought to life through a set that took

In [5]:

async def ask_question(ask: str):
    """
    Ask a question
    """
    
    # create a vectorized query based on the question
    vector = VectorizedQuery(vector=get_embedding(ask), k_nearest_neighbors=5, fields="vector")


    # create search client to retrieve movies from the vector store
    found_docs = list(search_client.search(
        search_text=None,
        query_type="semantic",
        semantic_configuration_name="movies-semantic-config",
        vector_queries=[vector],
        select=["title", "genre", "plot"],
        top=5
    ))

    # print the found documents and the field that were selected
    for doc in found_docs:
        print("Movie: {}".format(doc["title"]))
        print("Genre: {}".format(doc["genre"]))
        print("----------")


    found_docs_as_text = " "
    for elem in enumerate(found_docs, start=1):    
        found_docs_as_text += " "+ "Movie Title: {}".format(doc["title"]) +" "+ "Movie story: {}".format(doc["plot"])

    # augment the question with the found documents and ask the LLM to generate a response
    system_prompt = """You are an assistant to the user, you are given some context below. Please answer the query of the user with as few words as possible. Answer only with the correct information and be as short as possible. If the response is about estimation just answer with the number. If the response is about multiple choice just respond with the value strings from the options. If the response is true or false just say True or False. Do not end the response with a point."""

    parameters = [system_prompt, ' Context:', found_docs_as_text , ' Question:', ask]
    joined_parameters = ''.join(parameters)

    response = client.chat.completions.create(
            model = deployment_name,
            messages = [{"role" : "assistant", "content" : joined_parameters}],
        )

    print (response.choices[0].message.content)
    return response.choices[0].message.content

Use this snippet to try your method with several questions.

In [6]:

answer = await ask_question("How many actors were featured in The Smonger Games?")
print('Answer:', answer)


Movie: The Smonger Games
Genre: Action
----------
Movie: The Smoorgh Games
Genre: Science Fiction
----------
Movie: The Smry Games
Genre: Action
----------
Movie: The Smoorgh Games
Genre: Adventure
----------
Movie: The Smoorgh Games
Genre: Sci-Fi
----------
50
Answer: 50


In [7]:
answer = await ask_question("Does 'The Lost City' have any sequels planned? True or False")
print('Answer:', answer)

Movie: The Lost City
Genre: Adventure
----------
Movie: The Lost City
Genre: Adventure
----------
Movie: The Lost City
Genre: Adventure
----------
Movie: The Lost City
Genre: Adventure
----------
Movie: The Lost City
Genre: Adventure
----------
False
Answer: False
